In [23]:
# Load the dataset

dataset = []
with open('cat-facts.txt', 'r') as file:
  dataset = file.readlines()
  print(f'Loaded {len(dataset)} entries')



Loaded 150 entries


In [24]:
dataset

['On average, cats spend 2/3 of every day sleeping. That means a nine-year-old cat has been awake for only three years of its life.\n',
 'Unlike dogs, cats do not have a sweet tooth. Scientists believe this is due to a mutation in a key taste receptor.\n',
 'When a cat chases its prey, it keeps its head level. Dogs and humans bob their heads up and down.\n',
 'The technical term for a cat’s hairball is a “bezoar.”\n',
 'A group of cats is called a “clowder.”\n',
 'Female cats tend to be right pawed, while male cats are more often left pawed. Interestingly, while 90% of humans are right handed, the remaining 10% of lefties also tend to be male.\n',
 'A cat can’t climb head first down a tree because every claw on a cat’s paw points the same way. To get down from a tree, a cat must back down.\n',
 'Cats make about 100 different sounds. Dogs make only about 10.\n',
 'A cat’s brain is biologically more similar to a human brain than it is to a dog’s. Both humans and cats have identical regio

In [25]:
# Each element in the VECTOR_DB will be a tuple (chunk, embedding)
# The embedding is a list of floats, for example: [0.1, 0.04, -0.34, 0.21, ...]
VECTOR_DB = []

In [26]:
# Implement the retrieval system

EMBEDDING_MODEL = 'hf.co/CompendiumLabs/bge-base-en-v1.5-gguf'
LANGUAGE_MODEL = 'hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF'


In [27]:
import ollama

def add_chunk_to_database(chunk):
  embedding = ollama.embed(model=EMBEDDING_MODEL, input=chunk)['embeddings'][0]
  VECTOR_DB.append((chunk, embedding))

for i, chunk in enumerate(dataset):
  add_chunk_to_database(chunk)
  print(f'Added chunk {i+1}/{len(dataset)} to the database')


Added chunk 1/150 to the database
Added chunk 2/150 to the database
Added chunk 3/150 to the database
Added chunk 4/150 to the database
Added chunk 5/150 to the database
Added chunk 6/150 to the database
Added chunk 7/150 to the database
Added chunk 8/150 to the database
Added chunk 9/150 to the database
Added chunk 10/150 to the database
Added chunk 11/150 to the database
Added chunk 12/150 to the database
Added chunk 13/150 to the database
Added chunk 14/150 to the database
Added chunk 15/150 to the database
Added chunk 16/150 to the database
Added chunk 17/150 to the database
Added chunk 18/150 to the database
Added chunk 19/150 to the database
Added chunk 20/150 to the database
Added chunk 21/150 to the database
Added chunk 22/150 to the database
Added chunk 23/150 to the database
Added chunk 24/150 to the database
Added chunk 25/150 to the database
Added chunk 26/150 to the database
Added chunk 27/150 to the database
Added chunk 28/150 to the database
Added chunk 29/150 to the dat

In [28]:
VECTOR_DB[0]

('On average, cats spend 2/3 of every day sleeping. That means a nine-year-old cat has been awake for only three years of its life.\n',
 [-0.035898186,
  -0.023049803,
  0.04725364,
  -0.07919767,
  0.037882872,
  -0.014470321,
  0.07594751,
  0.05356503,
  -0.014032813,
  -0.0026459799,
  -0.020074267,
  -0.0072502014,
  -0.05742792,
  0.01165317,
  -0.046634767,
  0.041503523,
  0.086629055,
  0.009217399,
  -0.031057796,
  -0.029371524,
  0.0038406374,
  0.026922317,
  -0.026495898,
  -0.0007924138,
  0.050380543,
  -0.015898483,
  -0.010842919,
  -0.001441784,
  0.0020316988,
  -0.0132193705,
  0.036925543,
  -0.031370223,
  -0.03428765,
  0.0075680832,
  0.026818633,
  -0.034790028,
  -0.013016954,
  -0.033244263,
  0.013257519,
  0.031914033,
  -0.033585608,
  -0.012730343,
  -0.018642817,
  0.013430074,
  -0.030400636,
  -0.016690949,
  -0.0017663077,
  -0.013126982,
  0.02666575,
  0.015373555,
  -0.032529954,
  0.0016667363,
  0.005412661,
  -0.0051596537,
  -0.02098636,
  0.0

In [29]:
def cosine_similarity(a, b):
  dot_product = sum([x * y for x, y in zip(a, b)])
  norm_a = sum([x ** 2 for x in a]) ** 0.5
  norm_b = sum([x ** 2 for x in b]) ** 0.5
  return dot_product / (norm_a * norm_b)

In [30]:
def retrieve(query, top_n=3):
  query_embedding = ollama.embed(model=EMBEDDING_MODEL, input=query)['embeddings'][0]
  # temporary list to store (chunk, similarity) pairs
  similarities = []
  for chunk, embedding in VECTOR_DB:
    similarity = cosine_similarity(query_embedding, embedding)
    similarities.append((chunk, similarity))
  # sort by similarity in descending order, because higher similarity means more relevant chunks
  similarities.sort(key=lambda x: x[1], reverse=True)
  # finally, return the top N most relevant chunks
  return similarities[:top_n]

In [34]:
# Chatbot

input_query = input('Ask me a question: ')
retrieved_knowledge = retrieve(input_query)

print('Retrieved knowledge:')
for chunk, similarity in retrieved_knowledge:
  print(f' - (similarity: {similarity:.2f}) {chunk}')

instruction_prompt = f'''You are a helpful chatbot.
Use only the following pieces of context to answer the question. Don't make up any new information. 
{'\n'.join([f' - {chunk}' for chunk, similarity in retrieved_knowledge])}
'''
print(instruction_prompt)

stream = ollama.chat(
  model=LANGUAGE_MODEL,
  messages=[
    {'role': 'system', 'content': instruction_prompt},
    {'role': 'user', 'content': input_query},
  ],
  stream=True,
)

# print the response from the chatbot in real-time
print('Chatbot response:')
for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

Retrieved knowledge:
 - (similarity: 0.92) The temperature in Strezhevoy is -10 degrees celsius.
 - (similarity: 0.57) The normal body temperature of a cat is between 100.5 ° and 102.5 °F. A cat is sick if its temperature goes below 100 ° or above 103 °F.

 - (similarity: 0.55) In the 1930s, two Russian biologists discovered that color change in Siamese kittens depend on their body temperature. Siamese cats carry albino genes that work only when the body temperature is above 98° F. If these kittens are left in a very warm room, their points won’t darken and they will stay a creamy white.

You are a helpful chatbot.
Use only the following pieces of context to answer the question. Don't make up any new information. 
 - The temperature in Strezhevoy is -10 degrees celsius.
 - The normal body temperature of a cat is between 100.5 ° and 102.5 °F. A cat is sick if its temperature goes below 100 ° or above 103 °F.

 - In the 1930s, two Russian biologists discovered that color change in Siames

In [33]:
chunk = "The temperature in Strezhevoy is -10 degrees celsius."

add_chunk_to_database(chunk)